## Restaurants

In [1]:
import numpy as np
import tensorflow as tf
import scipy.sparse
import os
import time
import sys
import argparse
import itertools
from pathlib import Path

# ==========================
# Dataset Configuration
# ==========================
# Uncomment the dataset you want to use.
# DATASET = 'amazonbook'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'
# DATASET = 'ml-1m'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
# DATASET = 'yelp2018'
# DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Restaurants'
# DATASET = 'frappe'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
# DATASET = 'lastfm'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'

# ==========================
# Data Loader Definition
# ==========================
class LoadData(object):
    def __init__(self, DATA_ROOT):
        self.trainfile = os.path.join(DATA_ROOT, 'train.csv')
        self.testfile = os.path.join(DATA_ROOT, 'test.csv')
        self.user_field_M, self.item_field_M = self.get_length()
        print("user_field_M", self.user_field_M)
        print("item_field_M", self.item_field_M)
        print("Total fields", self.user_field_M + self.item_field_M)
        self.item_bind_M = self.bind_item()   # assigns an ID for each item feature combination
        self.user_bind_M = self.bind_user()   # assigns an ID for each user feature combination
        print("item_bind_M", len(self.binded_items.values()))
        print("user_bind_M", len(self.binded_users.values()))
        self.item_map_list = []
        for itemid in self.item_map.keys():
            self.item_map_list.append([int(feature) for feature in self.item_map[itemid].strip().split('-')])
        # Also include mapping for key 0 if it exists
        self.item_map_list.append([int(feature) for feature in self.item_map[0].strip().split('-')])
        self.user_positive_list = self.get_positive_list(self.trainfile)
        self.Train_data, self.Test_data = self.construct_data()
        self.user_train, self.item_train = self.get_train_instances()
        self.user_test = self.get_test()

    def get_length(self):
        length_user = 0
        length_item = 0
        with open(self.trainfile) as f:
            line = f.readline()
            while line:
                user_features = line.strip().split(',')[0].split('-')
                item_features = line.strip().split(',')[1].split('-')
                for uf in user_features:
                    feature = int(uf)
                    if feature > length_user:
                        length_user = feature
                for itf in item_features:
                    feature = int(itf)
                    if feature > length_item:
                        length_item = feature
                line = f.readline()
        return length_user + 1, length_item + 1

    def bind_item(self):
        self.binded_items = {}  # mapping from item feature string to an ID
        self.item_map = {}      # mapping from ID to item feature string
        self.bind_i(self.trainfile)
        self.bind_i(self.testfile)
        return len(self.binded_items)

    def bind_i(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_items)
            while line:
                features = line.strip().split(',')
                item_features = features[1]
                if item_features not in self.binded_items:
                    self.binded_items[item_features] = i
                    self.item_map[i] = item_features
                    i += 1
                line = f.readline()

    def bind_user(self):
        self.binded_users = {}  # mapping from user feature string to an ID
        self.user_map = {}      # mapping from ID to user feature string
        self.bind_u(self.trainfile)
        self.bind_u(self.testfile)
        return len(self.binded_users)

    def bind_u(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_users)
            while line:
                features = line.strip().split(',')
                user_features = features[0]
                if user_features not in self.binded_users:
                    self.binded_users[user_features] = i
                    self.user_map[i] = user_features
                    i += 1
                line = f.readline()

    def get_positive_list(self, file):
        self.max_positive_len = 0
        user_positive_list = {}
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_id = self.binded_users[features[0]]
                item_id = self.binded_items[features[1]]
                if user_id in user_positive_list:
                    user_positive_list[user_id].append(item_id)
                else:
                    user_positive_list[user_id] = [item_id]
                line = f.readline()
        for uid in user_positive_list:
            if len(user_positive_list[uid]) > self.max_positive_len:
                self.max_positive_len = len(user_positive_list[uid])
        return user_positive_list

    def get_train_instances(self):
        user_train, item_train = [], []
        for uid in self.user_positive_list:
            u_train = [int(feature) for feature in self.user_map[uid].strip().split('-')]
            user_train.append(u_train)
            temp = self.user_positive_list[uid][:]
            while len(temp) < self.max_positive_len:
                temp.append(self.item_bind_M)
            item_train.append(temp)
        user_train = np.array(user_train)
        item_train = np.array(item_train)
        return user_train, item_train

    def construct_data(self):
        X_user, X_item = self.read_data(self.trainfile)
        Train_data = self.construct_dataset(X_user, X_item)
        print("# of training samples:", len(X_user))
        X_user, X_item = self.read_data(self.testfile)
        Test_data = self.construct_dataset(X_user, X_item)
        print("# of test samples:", len(X_user))
        return Train_data, Test_data

    def construct_dataset(self, X_user, X_item):
        user_id = []
        for one in X_user:
            key = "-".join([str(item) for item in one])
            user_id.append(self.binded_users[key])
        item_id = []
        for one in X_item:
            key = "-".join([str(item) for item in one])
            item_id.append(self.binded_items[key])
        count = np.ones(len(X_user))
        sparse_matrix = scipy.sparse.csr_matrix((count, (user_id, item_id)),
                                                dtype=np.int16,
                                                shape=(self.user_bind_M, self.item_bind_M))
        return sparse_matrix

    def get_test(self):
        X_user, _ = self.read_data(self.testfile)
        return X_user

    def read_data(self, file):
        X_user = []
        X_item = []
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_features = features[0].split('-')
                X_user.append([int(item) for item in user_features])
                item_features = features[1].split('-')
                X_item.append([int(item) for item in item_features])
                line = f.readline()
        return X_user, X_item

# ==========================
# Model & Training Definitions
# ==========================
def parse_args():
    # For notebooks, we call parse_args with an empty list.
    parser = argparse.ArgumentParser(description="Run ENSFM with Hyperparameter Search")
    parser.add_argument('--dataset', default='yelp2018',
                        help='Dataset name: lastfm, frappe, ml-1m, yelp2018, amazonbook')
    parser.add_argument('--batch_size', type=int, default=512, help='Batch size')
    parser.add_argument('--epochs', type=int, default=50, help='Number of epochs for grid search')
    parser.add_argument('--verbose', type=int, default=10, help='Evaluation interval (in epochs)')
    parser.add_argument('--embed_size', type=int, default=64, help='Embedding size')
    parser.add_argument('--lr', type=float, default=0.05, help='Learning rate (overridden in grid search)')
    parser.add_argument('--dropout', type=float, default=0.9, help='Dropout keep probability (overridden)')
    parser.add_argument('--negative_weight', type=float, default=0.05, help='Negative weight (overridden)')
    parser.add_argument('--topK', type=int, nargs='+', default=[5, 10, 20], help='Top K for evaluation')
    return parser.parse_args([])

def _writeline_and_time(s):
    sys.stdout.write(s)
    sys.stdout.flush()
    return time.time()

class ENSFM:
    def __init__(self, item_attribute, user_field_M, item_field_M, embedding_size, max_item_pu, args):
        self.embedding_size = embedding_size
        self.max_item_pu = max_item_pu
        self.user_field_M = user_field_M
        self.item_field_M = item_field_M
        self.weight1 = args.negative_weight
        self.item_attribute = item_attribute
        self.lambda_bilinear = [0.0, 0.0]

    def _create_placeholders(self):
        self.input_u = tf.compat.v1.placeholder(tf.int32, [None, None], name="input_u_feature")
        self.input_ur = tf.compat.v1.placeholder(tf.int32, [None, self.max_item_pu], name="input_ur")
        self.dropout_keep_prob = tf.compat.v1.placeholder(tf.float32, name="dropout_keep_prob")

    def _create_variables(self):
        self.uidW = tf.Variable(tf.random.truncated_normal([self.user_field_M, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="uidW")
        self.iidW = tf.Variable(tf.random.truncated_normal([self.item_field_M+1, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="iidW")
        self.H_i = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_i")
        self.H_s = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_s")
        self.u_bias = tf.Variable(tf.random.truncated_normal([self.user_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="u_bias")
        self.i_bias = tf.Variable(tf.random.truncated_normal([self.item_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="i_bias")
        self.bias = tf.Variable(tf.constant(0.0), name="bias")

    def _create_vectors(self):
        # User embedding
        self.user_feature_emb = tf.nn.embedding_lookup(self.uidW, self.input_u)
        self.summed_user_emb = tf.reduce_sum(self.user_feature_emb, axis=1)  # [batch, embed_size]
        # Apply dropout using TF1.x style (keep_prob)
        self.H_i_drop = tf.compat.v1.nn.dropout(self.H_i, keep_prob=self.dropout_keep_prob)
        self.H_s_drop = tf.compat.v1.nn.dropout(self.H_s, keep_prob=self.dropout_keep_prob)
        # Item embedding
        self.all_item_feature_emb = tf.nn.embedding_lookup(self.iidW, self.item_attribute)
        self.summed_all_item_emb = tf.reduce_sum(self.all_item_feature_emb, axis=1)  # [batch, embed_size]
        # Cross terms
        self.user_cross = 0.5 * (tf.square(self.summed_user_emb) - tf.reduce_sum(tf.square(self.user_feature_emb), axis=1))
        self.item_cross = 0.5 * (tf.square(self.summed_all_item_emb) - tf.reduce_sum(tf.square(self.all_item_feature_emb), axis=1))
        # Compute scores (resulting in [batch, 1, 1])
        self.user_cross_score = tf.matmul(tf.expand_dims(self.user_cross, 1), self.H_s_drop)
        self.item_cross_score = tf.matmul(tf.expand_dims(self.item_cross, 1), self.H_s_drop)
        # Bias terms
        self.user_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.u_bias, self.input_u), axis=1)  # [batch, 1]
        self.item_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.i_bias, self.item_attribute), axis=1)  # [batch, 1]
        # Constant ones
        self.I_user = tf.ones([tf.shape(self.input_u)[0], 1])
        self.I_item = tf.ones([tf.shape(self.summed_all_item_emb)[0], 1])
        # Instead of concatenating rank-3 tensors, squeeze the extra dims so that:
        #   - summed_user_emb is [batch, embed_size]
        #   - user_cross_score becomes [batch, 1]
        #   - user_bias is [batch, 1]
        # Then concatenate along axis 1 to get [batch, embed_size+2]
        user_cross_score_squeezed = tf.squeeze(self.user_cross_score, axis=1)
        user_bias_squeezed = tf.squeeze(tf.expand_dims(self.user_bias, 1), axis=1)
        self.p_emb = tf.concat([self.summed_user_emb,
                                 user_cross_score_squeezed + user_bias_squeezed + self.bias,
                                 self.I_user],
                                axis=1)
        # Similarly for q_emb
        item_cross_score_squeezed = tf.squeeze(self.item_cross_score, axis=1)
        self.q_emb = tf.concat([self.summed_all_item_emb,
                                 self.I_item,
                                 item_cross_score_squeezed + tf.squeeze(tf.expand_dims(self.item_bias, 1), axis=1)],
                                axis=1)
        # Build H_i_emb by concatenating H_i_drop with two scalars so that its shape becomes [embed_size+2, 1]
        self.H_i_emb = tf.concat([self.H_i_drop, [[1.0]], [[1.0]]], axis=0)

    def _create_inference(self):
        self.pos_item = tf.nn.embedding_lookup(self.q_emb, self.input_ur)
        # Assumes that the global data object has attribute item_bind_M
        self.pos_num_r = tf.cast(tf.not_equal(self.input_ur, data.item_bind_M), tf.float32)
        self.pos_item = tf.einsum('ab,abc->abc', self.pos_num_r, self.pos_item)
        self.pos_r = tf.einsum('ac,abc->abc', self.p_emb, self.pos_item)
        self.pos_r = tf.einsum('ajk,kl->ajl', self.pos_r, self.H_i_emb)
        self.pos_r = tf.reshape(self.pos_r, [-1, self.max_item_pu])

    def _pre(self):
        dot = tf.einsum('ac,bc->abc', self.p_emb, self.q_emb)
        pre = tf.einsum('ajk,kl->aj', dot, self.H_i_emb)
        return pre

    def _create_loss(self):
        self.loss1 = self.weight1 * tf.reduce_sum(
            tf.reduce_sum(tf.reduce_sum(tf.einsum('ab,ac->abc', self.q_emb, self.q_emb), axis=0) *
                          tf.reduce_sum(tf.einsum('ab,ac->abc', self.p_emb, self.p_emb), axis=0) *
                          tf.matmul(self.H_i_emb, self.H_i_emb, transpose_b=True), axis=0), axis=0)
        self.loss1 += tf.reduce_sum((1.0 - self.weight1) * tf.square(self.pos_r) - 2.0 * self.pos_r)
        self.l2_loss0 = tf.nn.l2_loss(self.uidW)
        self.l2_loss1 = tf.nn.l2_loss(self.iidW)
        self.loss = self.loss1 + self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1
        self.reg_loss = self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1

    def _build_graph(self):
        self._create_placeholders()
        self._create_variables()
        self._create_vectors()
        self._create_inference()
        self._create_loss()
        self.pre = self._pre()

def train_step(sess, deep, train_op, u_batch, i_batch, args):
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_ur: i_batch,
        deep.dropout_keep_prob: args.dropout
    }
    _, loss, loss1, reg_loss = sess.run([train_op, deep.loss, deep.loss1, deep.reg_loss], feed_dict)
    return loss, loss1, reg_loss

def evaluate(sess, deep, data, args):
    eva_batch = 128
    recall_all = [[] for _ in range(len(args.topK))]
    ndcg_all = [[] for _ in range(len(args.topK))]
    user_features = data.user_test
    num_batches = int(np.ceil(len(user_features) / eva_batch))
    for batch_num in range(num_batches):
        start_index = batch_num * eva_batch
        end_index = min((batch_num + 1) * eva_batch, len(user_features))
        u_batch = user_features[start_index:end_index]
        batch_users = end_index - start_index
        feed_dict = { deep.input_u: u_batch, deep.dropout_keep_prob: 1.0 }
        pre = sess.run(deep.pre, feed_dict)
        pre = np.array(pre)
        pre = np.delete(pre, -1, axis=1)
        user_ids = [data.binded_users["-".join(map(str, one))] for one in u_batch]
        idx = np.zeros_like(pre, dtype=bool)
        idx[data.Train_data[user_ids].nonzero()] = True
        pre[idx] = -np.inf
        for i, k in enumerate(args.topK):
            idx_topk_part = np.argpartition(-pre, k, axis=1)[:, :k]
            pre_bin = np.zeros_like(pre, dtype=bool)
            pre_bin[np.arange(batch_users)[:, None], idx_topk_part] = True
            true_bin = np.zeros_like(pre, dtype=bool)
            true_bin[data.Test_data[user_ids].nonzero()] = True
            hits = (np.logical_and(true_bin, pre_bin).sum(axis=1)).astype(np.float32)
            recall_all[i].append(hits / np.minimum(k, true_bin.sum(axis=1)))
            topk_part = pre[np.arange(batch_users)[:, None], idx_topk_part]
            idx_part = np.argsort(-topk_part, axis=1)
            idx_topk = idx_topk_part[np.arange(batch_users)[:, None], idx_part]
            tp = np.log(2) / np.log(np.arange(2, k + 2))
            test_batch = data.Test_data[user_ids]
            DCG = (test_batch[np.arange(batch_users)[:, None], idx_topk].toarray() * tp).sum(axis=1)
            IDCG = np.array([(tp[:min(n, k)]).sum() for n in test_batch.getnnz(axis=1)])
            ndcg_all[i].append(DCG / IDCG)
    recalls = [np.mean(np.hstack(r)) for r in recall_all]
    ndcgs = [np.mean(np.hstack(n)) for n in ndcg_all]
    print("Evaluation Metrics:")
    for i, k in enumerate(args.topK):
        print(f"Top {k}: HR={recalls[i]:.4f}, NDCG={ndcgs[i]:.4f}")
    return recalls, ndcgs

def run_experiment(args, data, random_seed=2019):
    with tf.Graph().as_default():
        tf.compat.v1.set_random_seed(random_seed)
        session_conf = tf.compat.v1.ConfigProto()
        session_conf.gpu_options.allow_growth = True
        sess = tf.compat.v1.Session(config=session_conf)
        with sess.as_default():
            deep = ENSFM(data.item_map_list, data.user_field_M, data.item_field_M,
                         args.embed_size, data.max_positive_len, args)
            deep._build_graph()
            train_op = tf.compat.v1.train.AdagradOptimizer(
                learning_rate=args.lr, initial_accumulator_value=1e-8).minimize(deep.loss)
            sess.run(tf.compat.v1.global_variables_initializer())
            print("Initial evaluation:")
            evaluate(sess, deep, data, args)
            for epoch in range(args.epochs):
                print(f"Epoch {epoch}")
                shuffle_idx = np.random.permutation(len(data.user_train))
                data.user_train = data.user_train[shuffle_idx]
                data.item_train = data.item_train[shuffle_idx]
                num_batches = int(np.ceil(len(data.user_train) / args.batch_size))
                for batch_num in range(num_batches):
                    start_index = batch_num * args.batch_size
                    end_index = min((batch_num + 1) * args.batch_size, len(data.user_train))
                    u_batch = data.user_train[start_index:end_index]
                    i_batch = data.item_train[start_index:end_index]
                    train_step(sess, deep, train_op, u_batch, i_batch, args)
                if epoch % args.verbose == 0:
                    evaluate(sess, deep, data, args)
            print("Final evaluation:")
            recalls, ndcgs = evaluate(sess, deep, data, args)
    hr_index = args.topK.index(10) if 10 in args.topK else 0
    return recalls[hr_index], ndcgs[hr_index]

# ==========================
# Main Hyperparameter Search
# ==========================
if __name__ == '__main__':
    args = parse_args()
    # Optionally update DATA_ROOT based on args.dataset
    if args.dataset == 'lastfm':
        print("Loading LastFM data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'
    elif args.dataset == 'frappe':
        print("Loading Frappe data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
    elif args.dataset == 'ml-1m':
        print("Loading ML-1M data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
    elif args.dataset == 'yelp2018':
        print("Loading Yelp2018 data")
        DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Restaurants'
    elif args.dataset == 'amazonbook':
        print("Loading Amazon Book data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'

    # Load data
    data = LoadData(DATA_ROOT)

    # Define hyperparameter grid.
    lr_values = [0.005, 0.01, 0.02, 0.05]
    dropout_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
    neg_weight_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]

    results = []
    output_file = os.path.join(DATA_ROOT, 'ENSFM_hyperparam_results.txt')
    with open(output_file, 'w') as f_out:
        for lr, dropout, neg_weight in itertools.product(lr_values, dropout_values, neg_weight_values):
            print(f"\nRunning experiment with lr={lr}, dropout={dropout}, negative_weight={neg_weight}")
            f_out.write(f"Parameters: lr={lr}, dropout={dropout}, negative_weight={neg_weight}\n")
            args.lr = lr
            args.dropout = dropout
            args.negative_weight = neg_weight
            # Use fewer epochs for grid search
            args.epochs = 50
            hr, ndcg = run_experiment(args, data, random_seed=2019)
            results.append(((lr, dropout, neg_weight), (hr, ndcg)))
            f_out.write(f"Final HR@10: {hr:.4f}, NDCG@10: {ndcg:.4f}\n\n")
            f_out.flush()

    sorted_results = sorted(results, key=lambda x: x[1][0], reverse=True)
    print("\nSorted hyperparameter search results (by HR@10):")
    with open(output_file, 'a') as f_out:
        f_out.write("Sorted hyperparameter search results (by HR@10):\n")
        for combo, metrics in sorted_results:
            line = f"Parameters (lr, dropout, negative_weight): {combo} => HR@10: {metrics[0]:.4f}, NDCG@10: {metrics[1]:.4f}\n"
            print(line.strip())
            f_out.write(line)


2025-04-09 00:22:21.343942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744147341.355839   11579 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744147341.359300   11579 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744147341.369390   11579 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744147341.369402   11579 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744147341.369404   11579 computation_placer.cc:177] computation placer alr

Loading Yelp2018 data
user_field_M 60641
item_field_M 34858
Total fields 95499
item_bind_M 32857
user_bind_M 60640
# of training samples: 547491
# of test samples: 60640

Running experiment with lr=0.005, dropout=0.1, negative_weight=0.001
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


I0000 00:00:1744147350.249116   11579 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5599 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Initial evaluation:


I0000 00:00:1744147350.513073   11579 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


...

I0000 00:00:1744154679.484597   11579 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5599 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


Initial evaluation:
Evaluation Metrics:
Top 5: HR=0.0001, NDCG=0.0001
Top 10: HR=0.0002, NDCG=0.0001
Top 20: HR=0.0005, NDCG=0.0002
Epoch 0
Evaluation Metrics:
Top 5: HR=0.0037, NDCG=0.0024
Top 10: HR=0.0072, NDCG=0.0035
Top 20: HR=0.0136, NDCG=0.0050
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Evaluation Metrics:
Top 5: HR=0.0038, NDCG=0.0024
Top 10: HR=0.0077, NDCG=0.0036
Top 20: HR=0.0138, NDCG=0.0052
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Evaluation Metrics:
Top 5: HR=0.0038, NDCG=0.0024
Top 10: HR=0.0079, NDCG=0.0037
Top 20: HR=0.0137, NDCG=0.0051
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Evaluation Metrics:
Top 5: HR=0.0040, NDCG=0.0025
Top 10: HR=0.0077, NDCG=0.0037
Top 20: HR=0.0142, NDCG=0.0053
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Evaluation Metrics:
Top 5: HR=0.0043, NDCG=0.0026
Top 10: HR=

I0000 00:00:1744155246.054869   11579 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5599 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:08:00.0, compute capability: 8.9


Initial evaluation:
Evaluation Metrics:
Top 5: HR=0.0001, NDCG=0.0001
Top 10: HR=0.0002, NDCG=0.0001
Top 20: HR=0.0005, NDCG=0.0002
Epoch 0
Evaluation Metrics:
Top 5: HR=0.0032, NDCG=0.0019
Top 10: HR=0.0071, NDCG=0.0031
Top 20: HR=0.0133, NDCG=0.0047
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Evaluation Metrics:
Top 5: HR=0.0039, NDCG=0.0022
Top 10: HR=0.0077, NDCG=0.0035
Top 20: HR=0.0143, NDCG=0.0051
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Evaluation Metrics:
Top 5: HR=0.0043, NDCG=0.0024
Top 10: HR=0.0076, NDCG=0.0035
Top 20: HR=0.0141, NDCG=0.0051
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26


KeyboardInterrupt: 

## Food

In [ ]:
import numpy as np
import tensorflow as tf
import scipy.sparse
import os
import time
import sys
import argparse
import itertools
from pathlib import Path

# ==========================
# Dataset Configuration
# ==========================
# Uncomment the dataset you want to use.
# DATASET = 'amazonbook'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'
# DATASET = 'ml-1m'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
# DATASET = 'yelp2018'
# DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Restaurants'
# DATASET = 'frappe'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
# DATASET = 'lastfm'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'

# ==========================
# Data Loader Definition
# ==========================
class LoadData(object):
    def __init__(self, DATA_ROOT):
        self.trainfile = os.path.join(DATA_ROOT, 'train.csv')
        self.testfile = os.path.join(DATA_ROOT, 'test.csv')
        self.user_field_M, self.item_field_M = self.get_length()
        print("user_field_M", self.user_field_M)
        print("item_field_M", self.item_field_M)
        print("Total fields", self.user_field_M + self.item_field_M)
        self.item_bind_M = self.bind_item()   # assigns an ID for each item feature combination
        self.user_bind_M = self.bind_user()   # assigns an ID for each user feature combination
        print("item_bind_M", len(self.binded_items.values()))
        print("user_bind_M", len(self.binded_users.values()))
        self.item_map_list = []
        for itemid in self.item_map.keys():
            self.item_map_list.append([int(feature) for feature in self.item_map[itemid].strip().split('-')])
        # Also include mapping for key 0 if it exists
        self.item_map_list.append([int(feature) for feature in self.item_map[0].strip().split('-')])
        self.user_positive_list = self.get_positive_list(self.trainfile)
        self.Train_data, self.Test_data = self.construct_data()
        self.user_train, self.item_train = self.get_train_instances()
        self.user_test = self.get_test()

    def get_length(self):
        length_user = 0
        length_item = 0
        with open(self.trainfile) as f:
            line = f.readline()
            while line:
                user_features = line.strip().split(',')[0].split('-')
                item_features = line.strip().split(',')[1].split('-')
                for uf in user_features:
                    feature = int(uf)
                    if feature > length_user:
                        length_user = feature
                for itf in item_features:
                    feature = int(itf)
                    if feature > length_item:
                        length_item = feature
                line = f.readline()
        return length_user + 1, length_item + 1

    def bind_item(self):
        self.binded_items = {}  # mapping from item feature string to an ID
        self.item_map = {}      # mapping from ID to item feature string
        self.bind_i(self.trainfile)
        self.bind_i(self.testfile)
        return len(self.binded_items)

    def bind_i(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_items)
            while line:
                features = line.strip().split(',')
                item_features = features[1]
                if item_features not in self.binded_items:
                    self.binded_items[item_features] = i
                    self.item_map[i] = item_features
                    i += 1
                line = f.readline()

    def bind_user(self):
        self.binded_users = {}  # mapping from user feature string to an ID
        self.user_map = {}      # mapping from ID to user feature string
        self.bind_u(self.trainfile)
        self.bind_u(self.testfile)
        return len(self.binded_users)

    def bind_u(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_users)
            while line:
                features = line.strip().split(',')
                user_features = features[0]
                if user_features not in self.binded_users:
                    self.binded_users[user_features] = i
                    self.user_map[i] = user_features
                    i += 1
                line = f.readline()

    def get_positive_list(self, file):
        self.max_positive_len = 0
        user_positive_list = {}
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_id = self.binded_users[features[0]]
                item_id = self.binded_items[features[1]]
                if user_id in user_positive_list:
                    user_positive_list[user_id].append(item_id)
                else:
                    user_positive_list[user_id] = [item_id]
                line = f.readline()
        for uid in user_positive_list:
            if len(user_positive_list[uid]) > self.max_positive_len:
                self.max_positive_len = len(user_positive_list[uid])
        return user_positive_list

    def get_train_instances(self):
        user_train, item_train = [], []
        for uid in self.user_positive_list:
            u_train = [int(feature) for feature in self.user_map[uid].strip().split('-')]
            user_train.append(u_train)
            temp = self.user_positive_list[uid][:]
            while len(temp) < self.max_positive_len:
                temp.append(self.item_bind_M)
            item_train.append(temp)
        user_train = np.array(user_train)
        item_train = np.array(item_train)
        return user_train, item_train

    def construct_data(self):
        X_user, X_item = self.read_data(self.trainfile)
        Train_data = self.construct_dataset(X_user, X_item)
        print("# of training samples:", len(X_user))
        X_user, X_item = self.read_data(self.testfile)
        Test_data = self.construct_dataset(X_user, X_item)
        print("# of test samples:", len(X_user))
        return Train_data, Test_data

    def construct_dataset(self, X_user, X_item):
        user_id = []
        for one in X_user:
            key = "-".join([str(item) for item in one])
            user_id.append(self.binded_users[key])
        item_id = []
        for one in X_item:
            key = "-".join([str(item) for item in one])
            item_id.append(self.binded_items[key])
        count = np.ones(len(X_user))
        sparse_matrix = scipy.sparse.csr_matrix((count, (user_id, item_id)),
                                                dtype=np.int16,
                                                shape=(self.user_bind_M, self.item_bind_M))
        return sparse_matrix

    def get_test(self):
        X_user, _ = self.read_data(self.testfile)
        return X_user

    def read_data(self, file):
        X_user = []
        X_item = []
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_features = features[0].split('-')
                X_user.append([int(item) for item in user_features])
                item_features = features[1].split('-')
                X_item.append([int(item) for item in item_features])
                line = f.readline()
        return X_user, X_item

# ==========================
# Model & Training Definitions
# ==========================
def parse_args():
    # For notebooks, we call parse_args with an empty list.
    parser = argparse.ArgumentParser(description="Run ENSFM with Hyperparameter Search")
    parser.add_argument('--dataset', default='yelp2018',
                        help='Dataset name: lastfm, frappe, ml-1m, yelp2018, amazonbook')
    parser.add_argument('--batch_size', type=int, default=512, help='Batch size')
    parser.add_argument('--epochs', type=int, default=50, help='Number of epochs for grid search')
    parser.add_argument('--verbose', type=int, default=10, help='Evaluation interval (in epochs)')
    parser.add_argument('--embed_size', type=int, default=64, help='Embedding size')
    parser.add_argument('--lr', type=float, default=0.05, help='Learning rate (overridden in grid search)')
    parser.add_argument('--dropout', type=float, default=0.9, help='Dropout keep probability (overridden)')
    parser.add_argument('--negative_weight', type=float, default=0.05, help='Negative weight (overridden)')
    parser.add_argument('--topK', type=int, nargs='+', default=[5, 10, 20], help='Top K for evaluation')
    return parser.parse_args([])

def _writeline_and_time(s):
    sys.stdout.write(s)
    sys.stdout.flush()
    return time.time()

class ENSFM:
    def __init__(self, item_attribute, user_field_M, item_field_M, embedding_size, max_item_pu, args):
        self.embedding_size = embedding_size
        self.max_item_pu = max_item_pu
        self.user_field_M = user_field_M
        self.item_field_M = item_field_M
        self.weight1 = args.negative_weight
        self.item_attribute = item_attribute
        self.lambda_bilinear = [0.0, 0.0]

    def _create_placeholders(self):
        self.input_u = tf.compat.v1.placeholder(tf.int32, [None, None], name="input_u_feature")
        self.input_ur = tf.compat.v1.placeholder(tf.int32, [None, self.max_item_pu], name="input_ur")
        self.dropout_keep_prob = tf.compat.v1.placeholder(tf.float32, name="dropout_keep_prob")

    def _create_variables(self):
        self.uidW = tf.Variable(tf.random.truncated_normal([self.user_field_M, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="uidW")
        self.iidW = tf.Variable(tf.random.truncated_normal([self.item_field_M+1, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="iidW")
        self.H_i = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_i")
        self.H_s = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_s")
        self.u_bias = tf.Variable(tf.random.truncated_normal([self.user_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="u_bias")
        self.i_bias = tf.Variable(tf.random.truncated_normal([self.item_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="i_bias")
        self.bias = tf.Variable(tf.constant(0.0), name="bias")

    def _create_vectors(self):
        # User embedding
        self.user_feature_emb = tf.nn.embedding_lookup(self.uidW, self.input_u)
        self.summed_user_emb = tf.reduce_sum(self.user_feature_emb, axis=1)  # [batch, embed_size]
        # Apply dropout using TF1.x style (keep_prob)
        self.H_i_drop = tf.compat.v1.nn.dropout(self.H_i, keep_prob=self.dropout_keep_prob)
        self.H_s_drop = tf.compat.v1.nn.dropout(self.H_s, keep_prob=self.dropout_keep_prob)
        # Item embedding
        self.all_item_feature_emb = tf.nn.embedding_lookup(self.iidW, self.item_attribute)
        self.summed_all_item_emb = tf.reduce_sum(self.all_item_feature_emb, axis=1)  # [batch, embed_size]
        # Cross terms
        self.user_cross = 0.5 * (tf.square(self.summed_user_emb) - tf.reduce_sum(tf.square(self.user_feature_emb), axis=1))
        self.item_cross = 0.5 * (tf.square(self.summed_all_item_emb) - tf.reduce_sum(tf.square(self.all_item_feature_emb), axis=1))
        # Compute scores (resulting in [batch, 1, 1])
        self.user_cross_score = tf.matmul(tf.expand_dims(self.user_cross, 1), self.H_s_drop)
        self.item_cross_score = tf.matmul(tf.expand_dims(self.item_cross, 1), self.H_s_drop)
        # Bias terms
        self.user_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.u_bias, self.input_u), axis=1)  # [batch, 1]
        self.item_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.i_bias, self.item_attribute), axis=1)  # [batch, 1]
        # Constant ones
        self.I_user = tf.ones([tf.shape(self.input_u)[0], 1])
        self.I_item = tf.ones([tf.shape(self.summed_all_item_emb)[0], 1])
        # Instead of concatenating rank-3 tensors, squeeze the extra dims so that:
        #   - summed_user_emb is [batch, embed_size]
        #   - user_cross_score becomes [batch, 1]
        #   - user_bias is [batch, 1]
        # Then concatenate along axis 1 to get [batch, embed_size+2]
        user_cross_score_squeezed = tf.squeeze(self.user_cross_score, axis=1)
        user_bias_squeezed = tf.squeeze(tf.expand_dims(self.user_bias, 1), axis=1)
        self.p_emb = tf.concat([self.summed_user_emb,
                                 user_cross_score_squeezed + user_bias_squeezed + self.bias,
                                 self.I_user],
                                axis=1)
        # Similarly for q_emb
        item_cross_score_squeezed = tf.squeeze(self.item_cross_score, axis=1)
        self.q_emb = tf.concat([self.summed_all_item_emb,
                                 self.I_item,
                                 item_cross_score_squeezed + tf.squeeze(tf.expand_dims(self.item_bias, 1), axis=1)],
                                axis=1)
        # Build H_i_emb by concatenating H_i_drop with two scalars so that its shape becomes [embed_size+2, 1]
        self.H_i_emb = tf.concat([self.H_i_drop, [[1.0]], [[1.0]]], axis=0)

    def _create_inference(self):
        self.pos_item = tf.nn.embedding_lookup(self.q_emb, self.input_ur)
        # Assumes that the global data object has attribute item_bind_M
        self.pos_num_r = tf.cast(tf.not_equal(self.input_ur, data.item_bind_M), tf.float32)
        self.pos_item = tf.einsum('ab,abc->abc', self.pos_num_r, self.pos_item)
        self.pos_r = tf.einsum('ac,abc->abc', self.p_emb, self.pos_item)
        self.pos_r = tf.einsum('ajk,kl->ajl', self.pos_r, self.H_i_emb)
        self.pos_r = tf.reshape(self.pos_r, [-1, self.max_item_pu])

    def _pre(self):
        dot = tf.einsum('ac,bc->abc', self.p_emb, self.q_emb)
        pre = tf.einsum('ajk,kl->aj', dot, self.H_i_emb)
        return pre

    def _create_loss(self):
        self.loss1 = self.weight1 * tf.reduce_sum(
            tf.reduce_sum(tf.reduce_sum(tf.einsum('ab,ac->abc', self.q_emb, self.q_emb), axis=0) *
                          tf.reduce_sum(tf.einsum('ab,ac->abc', self.p_emb, self.p_emb), axis=0) *
                          tf.matmul(self.H_i_emb, self.H_i_emb, transpose_b=True), axis=0), axis=0)
        self.loss1 += tf.reduce_sum((1.0 - self.weight1) * tf.square(self.pos_r) - 2.0 * self.pos_r)
        self.l2_loss0 = tf.nn.l2_loss(self.uidW)
        self.l2_loss1 = tf.nn.l2_loss(self.iidW)
        self.loss = self.loss1 + self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1
        self.reg_loss = self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1

    def _build_graph(self):
        self._create_placeholders()
        self._create_variables()
        self._create_vectors()
        self._create_inference()
        self._create_loss()
        self.pre = self._pre()

def train_step(sess, deep, train_op, u_batch, i_batch, args):
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_ur: i_batch,
        deep.dropout_keep_prob: args.dropout
    }
    _, loss, loss1, reg_loss = sess.run([train_op, deep.loss, deep.loss1, deep.reg_loss], feed_dict)
    return loss, loss1, reg_loss

def evaluate(sess, deep, data, args):
    eva_batch = 128
    recall_all = [[] for _ in range(len(args.topK))]
    ndcg_all = [[] for _ in range(len(args.topK))]
    user_features = data.user_test
    num_batches = int(np.ceil(len(user_features) / eva_batch))
    for batch_num in range(num_batches):
        start_index = batch_num * eva_batch
        end_index = min((batch_num + 1) * eva_batch, len(user_features))
        u_batch = user_features[start_index:end_index]
        batch_users = end_index - start_index
        feed_dict = { deep.input_u: u_batch, deep.dropout_keep_prob: 1.0 }
        pre = sess.run(deep.pre, feed_dict)
        pre = np.array(pre)
        pre = np.delete(pre, -1, axis=1)
        user_ids = [data.binded_users["-".join(map(str, one))] for one in u_batch]
        idx = np.zeros_like(pre, dtype=bool)
        idx[data.Train_data[user_ids].nonzero()] = True
        pre[idx] = -np.inf
        for i, k in enumerate(args.topK):
            idx_topk_part = np.argpartition(-pre, k, axis=1)[:, :k]
            pre_bin = np.zeros_like(pre, dtype=bool)
            pre_bin[np.arange(batch_users)[:, None], idx_topk_part] = True
            true_bin = np.zeros_like(pre, dtype=bool)
            true_bin[data.Test_data[user_ids].nonzero()] = True
            hits = (np.logical_and(true_bin, pre_bin).sum(axis=1)).astype(np.float32)
            recall_all[i].append(hits / np.minimum(k, true_bin.sum(axis=1)))
            topk_part = pre[np.arange(batch_users)[:, None], idx_topk_part]
            idx_part = np.argsort(-topk_part, axis=1)
            idx_topk = idx_topk_part[np.arange(batch_users)[:, None], idx_part]
            tp = np.log(2) / np.log(np.arange(2, k + 2))
            test_batch = data.Test_data[user_ids]
            DCG = (test_batch[np.arange(batch_users)[:, None], idx_topk].toarray() * tp).sum(axis=1)
            IDCG = np.array([(tp[:min(n, k)]).sum() for n in test_batch.getnnz(axis=1)])
            ndcg_all[i].append(DCG / IDCG)
    recalls = [np.mean(np.hstack(r)) for r in recall_all]
    ndcgs = [np.mean(np.hstack(n)) for n in ndcg_all]
    print("Evaluation Metrics:")
    for i, k in enumerate(args.topK):
        print(f"Top {k}: HR={recalls[i]:.4f}, NDCG={ndcgs[i]:.4f}")
    return recalls, ndcgs

def run_experiment(args, data, random_seed=2019):
    with tf.Graph().as_default():
        tf.compat.v1.set_random_seed(random_seed)
        session_conf = tf.compat.v1.ConfigProto()
        session_conf.gpu_options.allow_growth = True
        sess = tf.compat.v1.Session(config=session_conf)
        with sess.as_default():
            deep = ENSFM(data.item_map_list, data.user_field_M, data.item_field_M,
                         args.embed_size, data.max_positive_len, args)
            deep._build_graph()
            train_op = tf.compat.v1.train.AdagradOptimizer(
                learning_rate=args.lr, initial_accumulator_value=1e-8).minimize(deep.loss)
            sess.run(tf.compat.v1.global_variables_initializer())
            print("Initial evaluation:")
            evaluate(sess, deep, data, args)
            for epoch in range(args.epochs):
                print(f"Epoch {epoch}")
                shuffle_idx = np.random.permutation(len(data.user_train))
                data.user_train = data.user_train[shuffle_idx]
                data.item_train = data.item_train[shuffle_idx]
                num_batches = int(np.ceil(len(data.user_train) / args.batch_size))
                for batch_num in range(num_batches):
                    start_index = batch_num * args.batch_size
                    end_index = min((batch_num + 1) * args.batch_size, len(data.user_train))
                    u_batch = data.user_train[start_index:end_index]
                    i_batch = data.item_train[start_index:end_index]
                    train_step(sess, deep, train_op, u_batch, i_batch, args)
                if epoch % args.verbose == 0:
                    evaluate(sess, deep, data, args)
            print("Final evaluation:")
            recalls, ndcgs = evaluate(sess, deep, data, args)
    hr_index = args.topK.index(10) if 10 in args.topK else 0
    return recalls[hr_index], ndcgs[hr_index]

# ==========================
# Main Hyperparameter Search
# ==========================
if __name__ == '__main__':
    args = parse_args()
    # Optionally update DATA_ROOT based on args.dataset
    if args.dataset == 'lastfm':
        print("Loading LastFM data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'
    elif args.dataset == 'frappe':
        print("Loading Frappe data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
    elif args.dataset == 'ml-1m':
        print("Loading ML-1M data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
    elif args.dataset == 'yelp2018':
        print("Loading Yelp2018 data")
        DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Food'
    elif args.dataset == 'amazonbook':
        print("Loading Amazon Book data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'

    # Load data
    data = LoadData(DATA_ROOT)

    # Define hyperparameter grid.
    lr_values = [0.005, 0.01, 0.02, 0.05]
    dropout_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
    neg_weight_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]

    results = []
    output_file = os.path.join(DATA_ROOT, 'ENSFM_hyperparam_results.txt')
    with open(output_file, 'w') as f_out:
        for lr, dropout, neg_weight in itertools.product(lr_values, dropout_values, neg_weight_values):
            print(f"\nRunning experiment with lr={lr}, dropout={dropout}, negative_weight={neg_weight}")
            f_out.write(f"Parameters: lr={lr}, dropout={dropout}, negative_weight={neg_weight}\n")
            args.lr = lr
            args.dropout = dropout
            args.negative_weight = neg_weight
            # Use fewer epochs for grid search
            args.epochs = 50
            hr, ndcg = run_experiment(args, data, random_seed=2019)
            results.append(((lr, dropout, neg_weight), (hr, ndcg)))
            f_out.write(f"Final HR@10: {hr:.4f}, NDCG@10: {ndcg:.4f}\n\n")
            f_out.flush()

    sorted_results = sorted(results, key=lambda x: x[1][0], reverse=True)
    print("\nSorted hyperparameter search results (by HR@10):")
    with open(output_file, 'a') as f_out:
        f_out.write("Sorted hyperparameter search results (by HR@10):\n")
        for combo, metrics in sorted_results:
            line = f"Parameters (lr, dropout, negative_weight): {combo} => HR@10: {metrics[0]:.4f}, NDCG@10: {metrics[1]:.4f}\n"
            print(line.strip())
            f_out.write(line)


## Shopping

In [ ]:
import numpy as np
import tensorflow as tf
import scipy.sparse
import os
import time
import sys
import argparse
import itertools
from pathlib import Path

# ==========================
# Dataset Configuration
# ==========================
# Uncomment the dataset you want to use.
# DATASET = 'amazonbook'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'
# DATASET = 'ml-1m'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
# DATASET = 'yelp2018'
# DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Restaurants'
# DATASET = 'frappe'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
# DATASET = 'lastfm'
# DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'

# ==========================
# Data Loader Definition
# ==========================
class LoadData(object):
    def __init__(self, DATA_ROOT):
        self.trainfile = os.path.join(DATA_ROOT, 'train.csv')
        self.testfile = os.path.join(DATA_ROOT, 'test.csv')
        self.user_field_M, self.item_field_M = self.get_length()
        print("user_field_M", self.user_field_M)
        print("item_field_M", self.item_field_M)
        print("Total fields", self.user_field_M + self.item_field_M)
        self.item_bind_M = self.bind_item()   # assigns an ID for each item feature combination
        self.user_bind_M = self.bind_user()   # assigns an ID for each user feature combination
        print("item_bind_M", len(self.binded_items.values()))
        print("user_bind_M", len(self.binded_users.values()))
        self.item_map_list = []
        for itemid in self.item_map.keys():
            self.item_map_list.append([int(feature) for feature in self.item_map[itemid].strip().split('-')])
        # Also include mapping for key 0 if it exists
        self.item_map_list.append([int(feature) for feature in self.item_map[0].strip().split('-')])
        self.user_positive_list = self.get_positive_list(self.trainfile)
        self.Train_data, self.Test_data = self.construct_data()
        self.user_train, self.item_train = self.get_train_instances()
        self.user_test = self.get_test()

    def get_length(self):
        length_user = 0
        length_item = 0
        with open(self.trainfile) as f:
            line = f.readline()
            while line:
                user_features = line.strip().split(',')[0].split('-')
                item_features = line.strip().split(',')[1].split('-')
                for uf in user_features:
                    feature = int(uf)
                    if feature > length_user:
                        length_user = feature
                for itf in item_features:
                    feature = int(itf)
                    if feature > length_item:
                        length_item = feature
                line = f.readline()
        return length_user + 1, length_item + 1

    def bind_item(self):
        self.binded_items = {}  # mapping from item feature string to an ID
        self.item_map = {}      # mapping from ID to item feature string
        self.bind_i(self.trainfile)
        self.bind_i(self.testfile)
        return len(self.binded_items)

    def bind_i(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_items)
            while line:
                features = line.strip().split(',')
                item_features = features[1]
                if item_features not in self.binded_items:
                    self.binded_items[item_features] = i
                    self.item_map[i] = item_features
                    i += 1
                line = f.readline()

    def bind_user(self):
        self.binded_users = {}  # mapping from user feature string to an ID
        self.user_map = {}      # mapping from ID to user feature string
        self.bind_u(self.trainfile)
        self.bind_u(self.testfile)
        return len(self.binded_users)

    def bind_u(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.binded_users)
            while line:
                features = line.strip().split(',')
                user_features = features[0]
                if user_features not in self.binded_users:
                    self.binded_users[user_features] = i
                    self.user_map[i] = user_features
                    i += 1
                line = f.readline()

    def get_positive_list(self, file):
        self.max_positive_len = 0
        user_positive_list = {}
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_id = self.binded_users[features[0]]
                item_id = self.binded_items[features[1]]
                if user_id in user_positive_list:
                    user_positive_list[user_id].append(item_id)
                else:
                    user_positive_list[user_id] = [item_id]
                line = f.readline()
        for uid in user_positive_list:
            if len(user_positive_list[uid]) > self.max_positive_len:
                self.max_positive_len = len(user_positive_list[uid])
        return user_positive_list

    def get_train_instances(self):
        user_train, item_train = [], []
        for uid in self.user_positive_list:
            u_train = [int(feature) for feature in self.user_map[uid].strip().split('-')]
            user_train.append(u_train)
            temp = self.user_positive_list[uid][:]
            while len(temp) < self.max_positive_len:
                temp.append(self.item_bind_M)
            item_train.append(temp)
        user_train = np.array(user_train)
        item_train = np.array(item_train)
        return user_train, item_train

    def construct_data(self):
        X_user, X_item = self.read_data(self.trainfile)
        Train_data = self.construct_dataset(X_user, X_item)
        print("# of training samples:", len(X_user))
        X_user, X_item = self.read_data(self.testfile)
        Test_data = self.construct_dataset(X_user, X_item)
        print("# of test samples:", len(X_user))
        return Train_data, Test_data

    def construct_dataset(self, X_user, X_item):
        user_id = []
        for one in X_user:
            key = "-".join([str(item) for item in one])
            user_id.append(self.binded_users[key])
        item_id = []
        for one in X_item:
            key = "-".join([str(item) for item in one])
            item_id.append(self.binded_items[key])
        count = np.ones(len(X_user))
        sparse_matrix = scipy.sparse.csr_matrix((count, (user_id, item_id)),
                                                dtype=np.int16,
                                                shape=(self.user_bind_M, self.item_bind_M))
        return sparse_matrix

    def get_test(self):
        X_user, _ = self.read_data(self.testfile)
        return X_user

    def read_data(self, file):
        X_user = []
        X_item = []
        with open(file) as f:
            line = f.readline()
            while line:
                features = line.strip().split(',')
                user_features = features[0].split('-')
                X_user.append([int(item) for item in user_features])
                item_features = features[1].split('-')
                X_item.append([int(item) for item in item_features])
                line = f.readline()
        return X_user, X_item

# ==========================
# Model & Training Definitions
# ==========================
def parse_args():
    # For notebooks, we call parse_args with an empty list.
    parser = argparse.ArgumentParser(description="Run ENSFM with Hyperparameter Search")
    parser.add_argument('--dataset', default='yelp2018',
                        help='Dataset name: lastfm, frappe, ml-1m, yelp2018, amazonbook')
    parser.add_argument('--batch_size', type=int, default=512, help='Batch size')
    parser.add_argument('--epochs', type=int, default=50, help='Number of epochs for grid search')
    parser.add_argument('--verbose', type=int, default=10, help='Evaluation interval (in epochs)')
    parser.add_argument('--embed_size', type=int, default=64, help='Embedding size')
    parser.add_argument('--lr', type=float, default=0.05, help='Learning rate (overridden in grid search)')
    parser.add_argument('--dropout', type=float, default=0.9, help='Dropout keep probability (overridden)')
    parser.add_argument('--negative_weight', type=float, default=0.05, help='Negative weight (overridden)')
    parser.add_argument('--topK', type=int, nargs='+', default=[5, 10, 20], help='Top K for evaluation')
    return parser.parse_args([])

def _writeline_and_time(s):
    sys.stdout.write(s)
    sys.stdout.flush()
    return time.time()

class ENSFM:
    def __init__(self, item_attribute, user_field_M, item_field_M, embedding_size, max_item_pu, args):
        self.embedding_size = embedding_size
        self.max_item_pu = max_item_pu
        self.user_field_M = user_field_M
        self.item_field_M = item_field_M
        self.weight1 = args.negative_weight
        self.item_attribute = item_attribute
        self.lambda_bilinear = [0.0, 0.0]

    def _create_placeholders(self):
        self.input_u = tf.compat.v1.placeholder(tf.int32, [None, None], name="input_u_feature")
        self.input_ur = tf.compat.v1.placeholder(tf.int32, [None, self.max_item_pu], name="input_ur")
        self.dropout_keep_prob = tf.compat.v1.placeholder(tf.float32, name="dropout_keep_prob")

    def _create_variables(self):
        self.uidW = tf.Variable(tf.random.truncated_normal([self.user_field_M, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="uidW")
        self.iidW = tf.Variable(tf.random.truncated_normal([self.item_field_M+1, self.embedding_size],
                                                             mean=0.0, stddev=0.01), name="iidW")
        self.H_i = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_i")
        self.H_s = tf.Variable(tf.constant(0.01, shape=[self.embedding_size, 1]), name="H_s")
        self.u_bias = tf.Variable(tf.random.truncated_normal([self.user_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="u_bias")
        self.i_bias = tf.Variable(tf.random.truncated_normal([self.item_field_M, 1],
                                                               mean=0.0, stddev=0.01), name="i_bias")
        self.bias = tf.Variable(tf.constant(0.0), name="bias")

    def _create_vectors(self):
        # User embedding
        self.user_feature_emb = tf.nn.embedding_lookup(self.uidW, self.input_u)
        self.summed_user_emb = tf.reduce_sum(self.user_feature_emb, axis=1)  # [batch, embed_size]
        # Apply dropout using TF1.x style (keep_prob)
        self.H_i_drop = tf.compat.v1.nn.dropout(self.H_i, keep_prob=self.dropout_keep_prob)
        self.H_s_drop = tf.compat.v1.nn.dropout(self.H_s, keep_prob=self.dropout_keep_prob)
        # Item embedding
        self.all_item_feature_emb = tf.nn.embedding_lookup(self.iidW, self.item_attribute)
        self.summed_all_item_emb = tf.reduce_sum(self.all_item_feature_emb, axis=1)  # [batch, embed_size]
        # Cross terms
        self.user_cross = 0.5 * (tf.square(self.summed_user_emb) - tf.reduce_sum(tf.square(self.user_feature_emb), axis=1))
        self.item_cross = 0.5 * (tf.square(self.summed_all_item_emb) - tf.reduce_sum(tf.square(self.all_item_feature_emb), axis=1))
        # Compute scores (resulting in [batch, 1, 1])
        self.user_cross_score = tf.matmul(tf.expand_dims(self.user_cross, 1), self.H_s_drop)
        self.item_cross_score = tf.matmul(tf.expand_dims(self.item_cross, 1), self.H_s_drop)
        # Bias terms
        self.user_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.u_bias, self.input_u), axis=1)  # [batch, 1]
        self.item_bias = tf.reduce_sum(tf.nn.embedding_lookup(self.i_bias, self.item_attribute), axis=1)  # [batch, 1]
        # Constant ones
        self.I_user = tf.ones([tf.shape(self.input_u)[0], 1])
        self.I_item = tf.ones([tf.shape(self.summed_all_item_emb)[0], 1])
        # Instead of concatenating rank-3 tensors, squeeze the extra dims so that:
        #   - summed_user_emb is [batch, embed_size]
        #   - user_cross_score becomes [batch, 1]
        #   - user_bias is [batch, 1]
        # Then concatenate along axis 1 to get [batch, embed_size+2]
        user_cross_score_squeezed = tf.squeeze(self.user_cross_score, axis=1)
        user_bias_squeezed = tf.squeeze(tf.expand_dims(self.user_bias, 1), axis=1)
        self.p_emb = tf.concat([self.summed_user_emb,
                                 user_cross_score_squeezed + user_bias_squeezed + self.bias,
                                 self.I_user],
                                axis=1)
        # Similarly for q_emb
        item_cross_score_squeezed = tf.squeeze(self.item_cross_score, axis=1)
        self.q_emb = tf.concat([self.summed_all_item_emb,
                                 self.I_item,
                                 item_cross_score_squeezed + tf.squeeze(tf.expand_dims(self.item_bias, 1), axis=1)],
                                axis=1)
        # Build H_i_emb by concatenating H_i_drop with two scalars so that its shape becomes [embed_size+2, 1]
        self.H_i_emb = tf.concat([self.H_i_drop, [[1.0]], [[1.0]]], axis=0)

    def _create_inference(self):
        self.pos_item = tf.nn.embedding_lookup(self.q_emb, self.input_ur)
        # Assumes that the global data object has attribute item_bind_M
        self.pos_num_r = tf.cast(tf.not_equal(self.input_ur, data.item_bind_M), tf.float32)
        self.pos_item = tf.einsum('ab,abc->abc', self.pos_num_r, self.pos_item)
        self.pos_r = tf.einsum('ac,abc->abc', self.p_emb, self.pos_item)
        self.pos_r = tf.einsum('ajk,kl->ajl', self.pos_r, self.H_i_emb)
        self.pos_r = tf.reshape(self.pos_r, [-1, self.max_item_pu])

    def _pre(self):
        dot = tf.einsum('ac,bc->abc', self.p_emb, self.q_emb)
        pre = tf.einsum('ajk,kl->aj', dot, self.H_i_emb)
        return pre

    def _create_loss(self):
        self.loss1 = self.weight1 * tf.reduce_sum(
            tf.reduce_sum(tf.reduce_sum(tf.einsum('ab,ac->abc', self.q_emb, self.q_emb), axis=0) *
                          tf.reduce_sum(tf.einsum('ab,ac->abc', self.p_emb, self.p_emb), axis=0) *
                          tf.matmul(self.H_i_emb, self.H_i_emb, transpose_b=True), axis=0), axis=0)
        self.loss1 += tf.reduce_sum((1.0 - self.weight1) * tf.square(self.pos_r) - 2.0 * self.pos_r)
        self.l2_loss0 = tf.nn.l2_loss(self.uidW)
        self.l2_loss1 = tf.nn.l2_loss(self.iidW)
        self.loss = self.loss1 + self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1
        self.reg_loss = self.lambda_bilinear[0] * self.l2_loss0 + self.lambda_bilinear[1] * self.l2_loss1

    def _build_graph(self):
        self._create_placeholders()
        self._create_variables()
        self._create_vectors()
        self._create_inference()
        self._create_loss()
        self.pre = self._pre()

def train_step(sess, deep, train_op, u_batch, i_batch, args):
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_ur: i_batch,
        deep.dropout_keep_prob: args.dropout
    }
    _, loss, loss1, reg_loss = sess.run([train_op, deep.loss, deep.loss1, deep.reg_loss], feed_dict)
    return loss, loss1, reg_loss

def evaluate(sess, deep, data, args):
    eva_batch = 128
    recall_all = [[] for _ in range(len(args.topK))]
    ndcg_all = [[] for _ in range(len(args.topK))]
    user_features = data.user_test
    num_batches = int(np.ceil(len(user_features) / eva_batch))
    for batch_num in range(num_batches):
        start_index = batch_num * eva_batch
        end_index = min((batch_num + 1) * eva_batch, len(user_features))
        u_batch = user_features[start_index:end_index]
        batch_users = end_index - start_index
        feed_dict = { deep.input_u: u_batch, deep.dropout_keep_prob: 1.0 }
        pre = sess.run(deep.pre, feed_dict)
        pre = np.array(pre)
        pre = np.delete(pre, -1, axis=1)
        user_ids = [data.binded_users["-".join(map(str, one))] for one in u_batch]
        idx = np.zeros_like(pre, dtype=bool)
        idx[data.Train_data[user_ids].nonzero()] = True
        pre[idx] = -np.inf
        for i, k in enumerate(args.topK):
            idx_topk_part = np.argpartition(-pre, k, axis=1)[:, :k]
            pre_bin = np.zeros_like(pre, dtype=bool)
            pre_bin[np.arange(batch_users)[:, None], idx_topk_part] = True
            true_bin = np.zeros_like(pre, dtype=bool)
            true_bin[data.Test_data[user_ids].nonzero()] = True
            hits = (np.logical_and(true_bin, pre_bin).sum(axis=1)).astype(np.float32)
            recall_all[i].append(hits / np.minimum(k, true_bin.sum(axis=1)))
            topk_part = pre[np.arange(batch_users)[:, None], idx_topk_part]
            idx_part = np.argsort(-topk_part, axis=1)
            idx_topk = idx_topk_part[np.arange(batch_users)[:, None], idx_part]
            tp = np.log(2) / np.log(np.arange(2, k + 2))
            test_batch = data.Test_data[user_ids]
            DCG = (test_batch[np.arange(batch_users)[:, None], idx_topk].toarray() * tp).sum(axis=1)
            IDCG = np.array([(tp[:min(n, k)]).sum() for n in test_batch.getnnz(axis=1)])
            ndcg_all[i].append(DCG / IDCG)
    recalls = [np.mean(np.hstack(r)) for r in recall_all]
    ndcgs = [np.mean(np.hstack(n)) for n in ndcg_all]
    print("Evaluation Metrics:")
    for i, k in enumerate(args.topK):
        print(f"Top {k}: HR={recalls[i]:.4f}, NDCG={ndcgs[i]:.4f}")
    return recalls, ndcgs

def run_experiment(args, data, random_seed=2019):
    with tf.Graph().as_default():
        tf.compat.v1.set_random_seed(random_seed)
        session_conf = tf.compat.v1.ConfigProto()
        session_conf.gpu_options.allow_growth = True
        sess = tf.compat.v1.Session(config=session_conf)
        with sess.as_default():
            deep = ENSFM(data.item_map_list, data.user_field_M, data.item_field_M,
                         args.embed_size, data.max_positive_len, args)
            deep._build_graph()
            train_op = tf.compat.v1.train.AdagradOptimizer(
                learning_rate=args.lr, initial_accumulator_value=1e-8).minimize(deep.loss)
            sess.run(tf.compat.v1.global_variables_initializer())
            print("Initial evaluation:")
            evaluate(sess, deep, data, args)
            for epoch in range(args.epochs):
                print(f"Epoch {epoch}")
                shuffle_idx = np.random.permutation(len(data.user_train))
                data.user_train = data.user_train[shuffle_idx]
                data.item_train = data.item_train[shuffle_idx]
                num_batches = int(np.ceil(len(data.user_train) / args.batch_size))
                for batch_num in range(num_batches):
                    start_index = batch_num * args.batch_size
                    end_index = min((batch_num + 1) * args.batch_size, len(data.user_train))
                    u_batch = data.user_train[start_index:end_index]
                    i_batch = data.item_train[start_index:end_index]
                    train_step(sess, deep, train_op, u_batch, i_batch, args)
                if epoch % args.verbose == 0:
                    evaluate(sess, deep, data, args)
            print("Final evaluation:")
            recalls, ndcgs = evaluate(sess, deep, data, args)
    hr_index = args.topK.index(10) if 10 in args.topK else 0
    return recalls[hr_index], ndcgs[hr_index]

# ==========================
# Main Hyperparameter Search
# ==========================
if __name__ == '__main__':
    args = parse_args()
    # Optionally update DATA_ROOT based on args.dataset
    if args.dataset == 'lastfm':
        print("Loading LastFM data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/lastfm'
    elif args.dataset == 'frappe':
        print("Loading Frappe data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/frappe'
    elif args.dataset == 'ml-1m':
        print("Loading ML-1M data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/ml-1m'
    elif args.dataset == 'yelp2018':
        print("Loading Yelp2018 data")
        DATA_ROOT = '/media/leo/Huy/Project/CARS/Yelp JSON/yelp_dataset/Shopping'
    elif args.dataset == 'amazonbook':
        print("Loading Amazon Book data")
        DATA_ROOT = '/content/drive/MyDrive/Record/data/amzbook'

    # Load data
    data = LoadData(DATA_ROOT)

    # Define hyperparameter grid.
    lr_values = [0.005, 0.01, 0.02, 0.05]
    dropout_values = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
    neg_weight_values = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0]

    results = []
    output_file = os.path.join(DATA_ROOT, 'ENSFM_hyperparam_results.txt')
    with open(output_file, 'w') as f_out:
        for lr, dropout, neg_weight in itertools.product(lr_values, dropout_values, neg_weight_values):
            print(f"\nRunning experiment with lr={lr}, dropout={dropout}, negative_weight={neg_weight}")
            f_out.write(f"Parameters: lr={lr}, dropout={dropout}, negative_weight={neg_weight}\n")
            args.lr = lr
            args.dropout = dropout
            args.negative_weight = neg_weight
            # Use fewer epochs for grid search
            args.epochs = 50
            hr, ndcg = run_experiment(args, data, random_seed=2019)
            results.append(((lr, dropout, neg_weight), (hr, ndcg)))
            f_out.write(f"Final HR@10: {hr:.4f}, NDCG@10: {ndcg:.4f}\n\n")
            f_out.flush()

    sorted_results = sorted(results, key=lambda x: x[1][0], reverse=True)
    print("\nSorted hyperparameter search results (by HR@10):")
    with open(output_file, 'a') as f_out:
        f_out.write("Sorted hyperparameter search results (by HR@10):\n")
        for combo, metrics in sorted_results:
            line = f"Parameters (lr, dropout, negative_weight): {combo} => HR@10: {metrics[0]:.4f}, NDCG@10: {metrics[1]:.4f}\n"
            print(line.strip())
            f_out.write(line)
